# Building an image detection server with gluoncv
In this notebook, we deploy a pre-trained image detection model
 1. We first import a model from the gluon model zoo locally on the notebook, that we then compress and send to S3
 1. We then use the SageMaker MXNet Serving feature to deploy the model to a real-time managed endpoint. It uses the model artifact that we previously loaded to S3.
 1. We query the endpoint and visualize detection results


* **More on `gluon` and `gluoncv`:**
 * [gluon](https://mxnet.incubator.apache.org/api/python/docs/api/gluon/index.html)** is the imperative python front-end of the Apache MXNet deep learning framework. Gluon notably features specialized toolkits helping reproducing state-of-the-art architectures: [gluon-cv](https://gluon-cv.mxnet.io/), [gluon-nlp](https://gluon-nlp.mxnet.io/), [gluon-ts](https://gluon-ts.mxnet.io/). Gluon also features a number of excellent end-to-end tutorial mixing science with code such as [D2L.ai](https://classic.d2l.ai/) and [The Straight Dope](https://gluon.mxnet.io/)
 * [gluoncv](https://gluon-cv.mxnet.io/contents.html) is an efficient computer vision toolkit written on top of `gluon` and MXNet aiming to make state-of-the-art vision research reproducible. 

* This specific demo has been developed on the `conda_mxnet_p36` kernel of a SageMaker `ml.c5.2xlarge` Notebook instance

**This sample is provided for demonstration purposes, make sure to conduct appropriate testing if derivating this code for your own use-cases!**

In [ ]:
! pip install -r requirements.txt

In [ ]:
import datetime
import json
import math
import os
import shutil
import subprocess as sb
import tarfile

import boto3
import gluoncv
from gluoncv import model_zoo, data, utils
from matplotlib import pyplot as plt
import mxnet as mx
from mxnet import gluon, image, nd

# import SageMaker tools
import sagemaker
from sagemaker import get_execution_role
from sagemaker.mxnet.model import MXNetModel

sm_sess = sagemaker.Session()

s3_bucket = sm_sess.default_bucket()  # We use this bucket to store model weights - don't hesitate to change.
print(f'using bucket {s3_bucket}')

sm_role = sm_sess.get_execution_role()

## Test a pre-trained detection model, locally
gluon model zoo https://mxnet.incubator.apache.org/api/python/gluon/model_zoo.html contains a variety of models.
In this demo we use a YoloV3 detection model (Redmon et Farhadi). More about YoloV3:
* paper https://pjreddie.com/media/files/papers/YOLOv3.pdf
* Website https://pjreddie.com/darknet/yolo/

Gluoncv model zoo contains a number of architectures with different performances in terms of speed and accuracy. If you are looking for speed or accuracy, don't hesitate to change the model

In [ ]:
model_name = 'yolo3_darknet53_coco'
net = model_zoo.get_model(model_name, pretrained=True)

The model we downloaded above is trained on the COCO dataset and can detect 80 classes. In this demo, we restrict the model to detect only specific classes of interest.
This idea is derived from the official gluoncv tutorial: https://gluon-cv.mxnet.io/build/examples_detection/skip_fintune.html


COCO contains the following classes:

In [ ]:
print('coco classes: ', net.classes)

In [ ]:
# in this demo we reset the detector to the "person" class

classes = ['person']

net.reset_class(classes=classes, reuse_weights=classes)
print('new classes: ', net.classes)

net.hybridize()  # hybridize to optimize computation

Test locally

In [ ]:
# gather couple public images
images = ['https://cdn.pixabay.com/photo/2017/06/09/05/17/person-2385787_960_720.jpg',
          'https://cdn.pixabay.com/photo/2019/11/04/01/11/cellular-4599956_960_720.jpg',
          #'https://cdn.pixabay.com/photo/2016/03/09/09/22/workplace-1245776_1280.jpg',
          'https://cdn.pixabay.com/photo/2017/07/31/21/04/people-2561053_1280.jpg',
          'https://cdn.pixabay.com/photo/2016/08/01/20/13/girl-1561989_1280.jpg',
          'https://cdn.pixabay.com/photo/2015/01/08/18/29/entrepreneur-593358_1280.jpg',
          #'https://cdn.pixabay.com/photo/2014/05/03/00/50/flower-child-336658_1280.jpg',
          'https://cdn.pixabay.com/photo/2014/07/31/23/49/guitarist-407212_1280.jpg',
          'https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Cyprus_national_football_team_2012.jpg/1600px-Cyprus_national_football_team_2012.jpg',
          'https://pixnio.com/free-images/2019/01/13/2019-01-13-09-46-22-1200x900.jpg',
          'https://cdn.pixabay.com/photo/2018/01/24/19/49/people-3104635_1280.jpg']

In [ ]:
# download the images locally

pic_folder = 'gluoncv-detector-pics'

sb.call(['mkdir', pic_folder])

shutil.rmtree(os.path.join(os.getcwd(), pic_folder))

for p in images:
    sb.call(['wget', p, '-P', pic_folder])

`gluoncv` comes with built-in pre-processing logic for popular detectors, including YoloV3:

https://gluon-cv.mxnet.io/_modules/gluoncv/data/transforms/presets/yolo.html

https://gluon-cv.mxnet.io/build/examples_detection/demo_yolo.html

In [ ]:
picnames = os.listdir(pic_folder)

n_pics = len(picnames)
n_cols = 3
n_rows = math.ceil(n_pics / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15,15))
[ax.axis("off") for ax_dim in axes for ax in ax_dim]
for i, pic in enumerate(picnames):
    curr_col = i % n_cols
    curr_row = i // n_cols
    # download and pre-process image
    print(pic)
    im_array =  image.imread(os.path.join(pic_folder, pic))
    x, orig_img = data.transforms.presets.yolo.transform_test(im_array)
    
    # forward pass and display
    box_ids, scores, bboxes = net(x)
    ax = utils.viz.plot_bbox(orig_img, bboxes[0], scores[0], box_ids[0], class_names=classes, thresh=0.9, ax=axes[curr_row, curr_col])
    ax.axis("off")
    ax.set_title(pic, pad=15)
fig.tight_layout()    
fig.show();

## Deploy the detection server
 1. We first need to **send the model to S3**, as we will provide the S3 model path to Amazon SageMaker endpoint creation API
 1. We create a **serving script** containing model deserialization code and inference logic
 1. We **deploy the endpoint** with a SageMaker SDK call

### save local model, compress and send to S3

In [ ]:
# save the full model (both weights and graph)
net.export(model_name, epoch=0)

In [ ]:
# compress
packname = 'model.tar.gz'
tar = tarfile.open(packname, 'w:gz')
tar.add('{}-symbol.json'.format(model_name))
tar.add('{}-0000.params'.format(model_name))
tar.close()

In [ ]:
# send to s3
s3_client = boto3.client('s3')
s3key = 'detection-artifact'
s3_client.upload_file(packname, s3_bucket, s3key + '/' + packname)

### Write the detection model in the SageMaker MXNet specification
https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/mxnet/README.rst#deploying-mxnet-models

Amazon SageMaker provides serving container for Sklearn, TensorFlow, PyTorch and Apache MXNet. This is convenient, because we don't have to write web server code: the server is already written, in the case of MXNet it is Multi Model Server ([MMS](https://github.com/awslabs/multi-model-server), also used to server PyTorch in SageMaker) . We just have to provide model deserialization code and serving logic.

The SageMaker MXNet model server breaks request handling into three steps. Each step involves invoking a python function, with information about the request and the return-value from the previous function in the chain:

* input processing, with `input_fn(request_body, request_content_type, model)`
* prediction, with `predict_fn(input_object, model)`
* output processing, with `output_fn(prediction, content_type)`

The full serving specification is documented here https://sagemaker.readthedocs.io/en/stable/using_mxnet.html#deploy-mxnet-models

In our specific example we don't write an `output_fn`, because `predict_fn` outputs an NDArray that can be handled to CSV or JSON by the default `output_fn`

In [ ]:
# we will store the script in the 'repo' directory
! mkdir repo

We create a requirements.txt to add an extra dependency to the SageMaker MXNet container (https://github.com/aws/sagemaker-mxnet-serving-container)

In [ ]:
%%writefile repo/requirements.txt
gluoncv==0.10.4.post4

In [ ]:
%%writefile repo/detection_server.py

import argparse
import ast
import logging
import os

from gluoncv import model_zoo, data, utils
import mxnet as mx
from mxnet import nd, gluon


def get_ctx():
    "function to get machine hardware context"
    try:
        _ = mx.nd.array([0], ctx=mx.gpu())
        ctx = mx.gpu()
    except:
        try:
            _ = mx.nd.array([0], ctx=mx.eia())
            ctx = mx.eia()
        except: 
            ctx = mx.cpu()
    return ctx


def model_fn(model_dir):
    """
    Load the gluon model. Called once when hosting service starts.
    :param: model_dir The directory where model files are stored.
    :return: a model (in this case a Gluon network)
    
    assumes that the parameters artifact is {model_name}.params
    """
    
    ctx = get_ctx()
    logging.info('Using ctx {}'.format(ctx))
    logging.info('Dir content {}'.format(os.listdir()))
    
    # instantiate net and reset to classes of interest
    net = gluon.nn.SymbolBlock.imports(
        symbol_file=[f for f in os.listdir() if f.endswith('json')][0],
        input_names=['data'],
        param_file=[f for f in os.listdir() if f.endswith('params')][0],
        ctx=ctx)
    
    return net


def input_fn(request_body, request_content_type):
    """prepares the input"""
        
    im_array = mx.image.imdecode(request_body)
    
    # Run YOLO pre-processing on CPU
    x, _ = data.transforms.presets.yolo.transform_test(im_array)
    logging.info('input_fn returns NDArray of shape ' + str(im_array.shape))
    
    return x


def predict_fn(input_object, model):
    """function used for prediction"""
    
    ctx = get_ctx()
    logging.info('Using ctx {}'.format(ctx))
    
    # forward pass and display
    box_ids, scores, bboxes = model(input_object.as_in_context(ctx))
    
    return nd.concat(box_ids, scores, bboxes, dim=2)  # return a single tensor

### Instantiate model and deploy

In [ ]:
model = MXNetModel(
    model_data='s3://{}/{}/{}'.format(s3_bucket, s3key, packname),
    role=sm_role,
    py_version='py37',
    entry_point='detection_server.py',
    source_dir='repo',
    framework_version='1.8.0')

We deploy to an ml.g4dn.xlarge instance featuring a T4 NVIDIA GPU

In [ ]:
endpoint_key = ((model_name + '-detection').replace('_', '-').replace('.', '') + '-' 
                + datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))

In [ ]:
%%time
import sagemaker.serializers
import sagemaker.deserializers


# this may take 5 to 10min
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge',
    endpoint_name=endpoint_key,
    serializer=sagemaker.serializers.JSONSerializer,
    deserializer=sagemaker.deserializers.JSONDeserializer)

In [ ]:
predictor.deserializer


In [ ]:
predictor.serializer

## Submit requests to the detection server

In [ ]:
#predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_key, serializer=sagemaker.serializers.JSONSerializer, deserializer=sagemaker.deserializers.JSONDeserializer)
predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_key)

In [ ]:
predictor.serializer

In [ ]:
predictor.deserializer

In [ ]:
predictor.accept

In [ ]:
def detect(pic, predictor):
    """elementary function to send a picture to a predictor"""
    
    with open(pic, 'rb') as f:
        image = f.read()
    #print(image)
    tensor = nd.array(json.loads(predictor.predict(image)))
    box_ids, scores, bboxes = tensor[:,:,0], tensor[:,:,1], tensor[:,:,2:]
    return box_ids, scores, bboxes

In [ ]:
%%time

# test

pic = 'girl-1561989_1280.jpg'

box_ids, scores, bboxes = detect(os.path.join(pic_folder, pic), predictor)

In [ ]:
# for local viz we need to resize local pic to the server-side resize
_, orig_img = data.transforms.presets.yolo.load_test(os.path.join(pic_folder, pic))
utils.viz.plot_bbox(orig_img, bboxes[0], scores[0], box_ids[0], class_names=classes, thresh=0.9)

Unsuprisingly this works as well! The main difference with the first demo done locally at the top of the notebook is that we now have a permanently up, remote ML model server reachable with API, along with server and hardware monitoring available in Cloudwatch

### Don't forget to delete the endpoint after the demo!

In [ ]:
sm_sess.delete_endpoint(endpoint_key)